#DATA AND PROBLEM

In [220]:
import numpy as np
import random
import time

In [221]:
class Problem:
    def random_solution(self) -> list:
        return []

    def evaluate(self, solution) -> float:
        return 0.0

In [222]:
class HASHCODE_PROBLEM(Problem):
    def __init__(self, days, number_of_libraries, libraries, books):
        self.days = days
        self.number_of_libraries = number_of_libraries
        self.libraries = libraries
        self.books = books

    def random_solution(self):
        solution = []
        days_left = self.days
        for library in np.random.permutation(self.libraries):
            new_library = Library(library.id, library.number_of_books, library.books, library.signup_time, library.books_per_day)
            days_left = max(days_left - library.signup_time, 0)
            new_library.days_left = days_left
            new_books = new_library.choose_random_books()
            solution.append((new_library, new_books))
        return solution

    def evaluate(self, solution) -> int:
        set_of_books = set()
        for library, books in solution:
            set_of_books.update(books)
        return sum(book.score for book in set_of_books)

In [223]:
class SolutionEvolutionary():
    def __init__(self, problem, size_of_population = 10, mutation_probability = 0.4, crossover_probability = 0.2, tournament_size = 0.2):
        self.problem = problem
        self.mutation_probability = mutation_probability
        self.crossover_probability = crossover_probability
        self.tournament_size = tournament_size
        self.population_size = size_of_population
        self.population = self.initialize_greedy_population()
        self.calculate_unique_books()
    
    def calculate_unique_books(self):
        for library in self.problem.libraries:
            unique_books = set(library.books)
            for library_check in self.problem.libraries:
                if library_check.id != library.id:
                    unique_books -= library_check.books
            library.unique_books = unique_books

    def initialize_population(self):
        population = []
        while len(population) < self.population_size:
            solution = self.problem.random_solution()
            population.append((solution, self.problem.evaluate(solution)))
        return population
    
    def omega(self, library):
        return library.books_per_day*sum([book.score for book in library.books])/library.number_of_books
    
    def fi(self, library):
        return sum([book.score for book in library.books])/library.signup_time

    def gamma(self, library):
        return 1/library.signup_time

    def epsilon(self, library):
        return sum([book.score for book in library.unique_books]) + sum([book.score for book in (library.books - library.unique_books)]) * library.books_per_day / max(len(library.books - library.unique_books), 1)

    def initialize_greedy_population(self):
        population = []
        NEWLIBRARIES = sorted(self.problem.libraries, key = lambda x: (self.epsilon(x)), reverse = True)
        while len(population) < self.population_size:
            solution = []
            read = set()
            days_left = self.problem.days
            for library in NEWLIBRARIES:
                new_library = Library(library.id, library.number_of_books, library.books, library.signup_time, library.books_per_day)
                days_left = max(days_left - library.signup_time, 0)
                new_library.days_left = days_left
                new_books = new_library.choose_greedy_books(read)
                solution.append((new_library, new_books))
            population.append((solution, self.problem.evaluate(solution)))
        return population

    def mutate(self, solution):
        if np.random.uniform(0, 1) > self.mutation_probability:
            return solution
        
        for i in range(self.problem.number_of_libraries // 2):
            if np.random.uniform(0, 1) <= self.mutation_probability / self.problem.number_of_libraries * 10:
                index1, index2 = random.sample(range(len(solution)), 2)
                solution[index1], solution[index2] = solution[index2], solution[index1]

        days_checker = self.problem.days
        new_solution = []
        for library, books in solution:
            days_checker = max(days_checker - library.signup_time, 0)
            library.days_left = days_checker
            new_books = library.fix_books(books)

            if np.random.uniform(0, 1) <= self.mutation_probability:
                new_books = library.mutate_books(new_books)
            new_solution.append((library, new_books))
        
        return new_solution
    
    def crossover_books(self, s1, s2, dictionary):

        new_solution = []
        days_checker = self.problem.days
            
        for library1, books1 in s1:
            if np.random.uniform(0, 1) > self.crossover_probability:
                days_checker = max(days_checker - library1.signup_time, 0)
                library1.days_left = days_checker
                new_books = library1.fix_books(books1)
                new_solution.append((library1, new_books))
                continue
            
            library2, books2 = s2[dictionary[library1.id]]
            intersection = books1.intersection(books2)

            n1 = len(books1) - len(intersection)
            n2 = len(books2) - len(intersection)

            if n2 >= n1//2:
                intersection.update(random.sample((books1-intersection), n1//2) + random.sample((books2-intersection), n1//2))

                days_checker = max(days_checker - library1.signup_time, 0)
                library1.days_left = days_checker
                new_books = library1.fix_books(intersection)

                new_solution.append((library1, new_books))    
            else:
                difference = n1 - n2
                intersection.update(random.sample((books1-intersection), difference) + random.sample((books2-intersection), n2))

                days_checker = max(days_checker - library1.signup_time, 0)
                library1.days_left = days_checker
                new_books = library1.fix_books(intersection)

                new_solution.append((library1, new_books))
        return new_solution

    def crossover(self, s1, s2):
        if np.random.uniform(0, 1) > self.crossover_probability:
            return s1, s2

        solution1 = []
        solution2 = []
        solution1_ids = set()
        dict1 = dict()
        dict2 = dict()

        for index, ((l1, b1), (l2, b2)) in enumerate(zip(s1, s2)):
            if l1.id not in solution1_ids:
                solution1.append((l1, b1))
                solution1_ids.add(l1.id)
                dict1[l1.id] = len(solution1) - 1
            else:
                solution2.append((l1, b1))
                dict2[l1.id] = len(solution2) - 1

            if l2.id not in solution1_ids:
                solution1.append((l2, b2))
                solution1_ids.add(l2.id)
                dict1[l2.id] = len(solution1) - 1
            else:
                solution2.append((l2, b2))
                dict2[l2.id] = len(solution2) - 1
        
        new_solution1 = self.crossover_books(solution1, solution2, dict2)
        new_solution2 = self.crossover_books(solution2, solution1, dict1)

        return new_solution1, new_solution2
    
    def copy_solution(self, solution):
        new_solution = []
        for library, books in solution:
            new_library = Library(library.id, library.number_of_books, library.books, library.signup_time, library.books_per_day)
            new_library.days_left = library.days_left
            new_books = set(books)
            new_solution.append((new_library, new_books))
        return new_solution

    def tournament_round(self, population, number_of_winners = 1):
        tournament = random.sample(population, max(1, int(self.tournament_size * self.population_size)))
        winner = self.copy_solution(tournament[0][0])
        return winner

    def evolution_step(self):
        winners = []
        for i in range(int(self.tournament_size * self.population_size)):
            solution = (self.copy_solution(self.population[i][0]), self.population[i][1])
            winners.append(solution)

        while len(winners) < self.population_size:
            winner1, winner2 = self.tournament_round(self.population), self.tournament_round(self.population)
            winner1, winner2 = self.crossover(winner1, winner2)
            winner1 = self.mutate(winner1)
            winner2 = self.mutate(winner2)

            winners.append((winner1, self.problem.evaluate(winner1)))
            winners.append((winner2, self.problem.evaluate(winner2)))
        
        winners = sorted(winners, key=lambda x: x[1], reverse = True)
        return winners

    def evolution(self, maximum, start_time, best_solution = None, execution_time = 0, no_improvement = 0):
        return self.population[0][1], self.population[0][0], time.time() - start_time
        for epoch, _ in enumerate(iter(bool, True)):
            return self.population[0][1], self.population[0][0], time.time() - start_time

            if execution_time > 240 or no_improvement > 480:
                return maximum, best_solution, execution_time
            
            self.population = self.evolution_step()

            if self.population[0][1] > maximum:
                no_improvement = 0
                best_solution = self.population[0][0]
                maximum = self.population[0][1]
            
            no_improvement += 1
            execution_time = time.time() - start_time

In [224]:
class Library:

    days_left = None
    unique_books = set()

    def __init__(self, id, number_of_books, books, signup_time, books_per_day):
        self.id = id
        self.number_of_books = number_of_books
        self.books = books
        self.signup_time = signup_time
        self.books_per_day = books_per_day

    def choose_random_books(self):
        if self.days_left == 0:
            return set()
        books = set(random.sample(self.books, min(self.number_of_books, self.books_per_day * self.days_left)))
        return books

    def choose_greedy_books(self, read):
        chosen = set()
        if self.days_left == 0:
            return set()

        books = sorted(list(self.books), key = lambda x: x.score, reverse = True)
        for book in books:
            if len(chosen) == self.days_left*self.books_per_day:
                return chosen
                
            if book not in read:
                chosen.add(book)
                read.add(book)
        return chosen

    
    def mutate_books(self, chosen_books):
        if len(chosen_books) == self.number_of_books or len(chosen_books) == 0:
            return chosen_books
        
        books_to_choose_from = self.books - chosen_books
        n = np.random.randint(1, min(len(books_to_choose_from), len(chosen_books))+1)

        books_to_remove = random.sample(chosen_books, n)
        books_to_add = random.sample(books_to_choose_from, n)

        chosen_books.difference_update(books_to_remove)
        chosen_books.update(books_to_add)        
        return chosen_books

    def fix_books(self, chosen_books):
        if self.days_left == 0:
            return set()
        
        delta = min(self.days_left * self.books_per_day, self.number_of_books) - len(chosen_books)

        if delta > 0:
            chosen_books.update(random.sample(self.books - chosen_books, delta))
            return chosen_books
        
        if delta < 0:
            chosen_books.difference_update(random.sample(chosen_books, -delta))
            return chosen_books
        
        return chosen_books

In [225]:
class Book:
    def __init__(self, id, score):
        self.id = id
        self.score = score

In [226]:
def write_to_file(file, solution):
    file.write(str(NUMBER_OF_LIBRARIES))
    file.write('\n')
    for library, books in solution:
        file.write(str(library.id))
        file.write(" ")
        file.write(str(len(books)))
        file.write('\n')
        books = " ".join(str(book.id) for book in books)
        file.write(books)

        file.write('\n')

In [227]:
names = ['a_example', 'b_read_on', 'c_incunabula', 'd_tough_choices', 'e_so_many_books', 'f_libraries_of_the_world']
EVALUATION = 0
start_time = time.time()
for index, name in enumerate(names):
    with open(name + ".txt", "r") as f_in:
        lines = f_in.read().splitlines()

    NUMBER_OF_BOOKS, NUMBER_OF_LIBRARIES, NUMBER_OF_DAYS = (int(x) for x in lines.pop(0).split())
    BOOKS = tuple(Book(int(id), int(score)) for id, score in enumerate(lines.pop(0).split()))

    LIBRARIES = []
    for library in range(NUMBER_OF_LIBRARIES):
        number_of_books, signup_time, books_per_day = (int(x) for x in lines.pop(0).split())
        books = set(BOOKS[int(id)] for id in lines.pop(0).split())
        LIBRARIES.append(Library(library, number_of_books, books, signup_time, books_per_day))

    HASH_CODE = HASHCODE_PROBLEM(NUMBER_OF_DAYS, NUMBER_OF_LIBRARIES, LIBRARIES, books)
    SOLUTION = SolutionEvolutionary(HASH_CODE, 1, 0.6, 0.2, 0.25)
    EVAL, best_solution, execution_time = SOLUTION.evolution(0, start_time)
    print(name, '=', EVAL)
    EVALUATION += EVAL
print('time:', time.time() - start_time)
print(EVALUATION)

a_example = 21
b_read_on = 4126100
c_incunabula = 1132105
d_tough_choices = 4109300
e_so_many_books = 647814
f_libraries_of_the_world = 104370
time: 703.104647397995
10119710


#idk

In [228]:
names = ['a_example', 'b_read_on', 'c_incunabula', 'd_tough_choices', 'e_so_many_books', 'f_libraries_of_the_world']
best_parameters = []
best_parameters_for_file = [(0, []) for i in range(len(names))]
best_evaluation = 0
for pop_size in [16, 32]:
    for mut_prob in [0.2, 0.4, 0.6]:
        for cross_prob in [0.2, 0.4, 0.6]:
            for tournament_size in [1/16, 1/8, 1/4]:
                EVALUATION = 0
                for index, name in enumerate(names):
                    start_time = time.time()
                    with open(name + ".txt", "r") as f_in:
                        lines = f_in.read().splitlines()

                    NUMBER_OF_BOOKS, NUMBER_OF_LIBRARIES, NUMBER_OF_DAYS = (int(x) for x in lines.pop(0).split())
                    BOOKS = tuple(Book(int(id), int(score)) for id, score in enumerate(lines.pop(0).split()))

                    LIBRARIES = []
                    for library in range(NUMBER_OF_LIBRARIES):
                        number_of_books, signup_time, books_per_day = (int(x) for x in lines.pop(0).split())
                        books = set(BOOKS[int(id)] for id in lines.pop(0).split())
                        LIBRARIES.append(Library(library, number_of_books, books, signup_time, books_per_day))

                    HASH_CODE = HASHCODE_PROBLEM(NUMBER_OF_DAYS, NUMBER_OF_LIBRARIES, LIBRARIES, books)
                    SOLUTION = SolutionEvolutionary(HASH_CODE, pop_size, mut_prob, cross_prob, tournament_size)
                    EVAL, best_solution, execution_time = SOLUTION.evolution(0, start_time)
                    if EVAL > best_parameters_for_file[index][0]:
                        best_parameters_for_file[index] = (EVAL, [pop_size, mut_prob, cross_prob, tournament_size])
                        print(name, ': ', best_parameters_for_file[index], execution_time)
                    EVALUATION += EVAL
                    """
                    print(name, ': ', best_evaluation)

                    with open(name + "_out" + ".txt", 'w') as f_out:
                        write_to_file(f_out, best_solution)
                    """
                if EVALUATION > best_evaluation:
                    best_evaluation = EVALUATION
                    best_parameters = [pop_size, mut_prob, cross_prob, tournament_size]
                    print(best_evaluation, best_parameters)

a_example :  (21, [16, 0.2, 0.2, 0.0625]) 0.005570888519287109
b_read_on :  (4126100, [16, 0.2, 0.2, 0.0625]) 1.7153334617614746
c_incunabula :  (1132105, [16, 0.2, 0.2, 0.0625]) 80.52980756759644
d_tough_choices :  (4109300, [16, 0.2, 0.2, 0.0625]) 512.0640125274658
e_so_many_books :  (647814, [16, 0.2, 0.2, 0.0625]) 15.064243078231812
f_libraries_of_the_world :  (104370, [16, 0.2, 0.2, 0.0625]) 17.20014190673828
10119710 [16, 0.2, 0.2, 0.0625]


KeyboardInterrupt: ignored

In [ ]:
a_example :  (21, [4, 0.2, 0.2, 0.0625]) 0.010535478591918945
b_read_on :  (5822900, [4, 0.2, 0.2, 0.0625]) 12.064156293869019
c_incunabula :  (5645747, [4, 0.2, 0.2, 0.0625]) 91.27921557426453
d_tough_choices :  (4815460, [4, 0.2, 0.2, 0.0625]) 121.32832956314087
e_so_many_books :  (4601955, [4, 0.2, 0.2, 0.0625]) 15.614769220352173
f_libraries_of_the_world :  (5240161, [4, 0.2, 0.2, 0.0625]) 14.042518377304077
26126244 [4, 0.2, 0.2, 0.0625]
c_incunabula :  (5646152, [4, 0.2, 0.2, 0.125]) 88.37913179397583
e_so_many_books :  (4602109, [4, 0.2, 0.2, 0.125]) 14.211727857589722
26126738 [4, 0.2, 0.2, 0.125]
c_incunabula :  (5646233, [4, 0.2, 0.4, 0.125]) 120.6269805431366
e_so_many_books :  (4602163, [4, 0.2, 0.4, 0.125]) 17.335426807403564
26126938 [4, 0.2, 0.4, 0.125]
c_incunabula :  (5646625, [4, 0.2, 0.4, 0.25]) 120.561350107193
d_tough_choices :  (4815720, [4, 0.2, 0.4, 0.25]) 121.68053698539734
26127510 [4, 0.2, 0.4, 0.25]
e_so_many_books :  (4602199, [4, 0.2, 0.6, 0.125]) 18.612205028533936
c_incunabula :  (5648317, [4, 0.2, 0.6, 0.25]) 120.80544018745422
e_so_many_books :  (4602245, [4, 0.2, 0.6, 0.25]) 20.88402247428894
26129104 [4, 0.2, 0.6, 0.25]
e_so_many_books :  (4602442, [4, 0.4, 0.2, 0.125]) 15.763833999633789
f_libraries_of_the_world :  (5253903, [4, 0.4, 0.4, 0.25]) 37.04763054847717
26141710 [4, 0.4, 0.4, 0.25]
c_incunabula :  (5649790, [4, 0.4, 0.6, 0.25]) 120.43827939033508
e_so_many_books :  (4602452, [4, 0.6, 0.4, 0.0625]) 19.42077660560608
c_incunabula :  (5652916, [4, 0.6, 0.6, 0.25]) 120.38840389251709
f_libraries_of_the_world :  (5261450, [8, 0.6, 0.4, 0.125]) 71.30715537071228
26149916 [8, 0.6, 0.4, 0.125]
d_tough_choices :  (4815785, [8, 0.6, 0.4, 0.25]) 126.17268443107605
f_libraries_of_the_world :  (5268525, [8, 0.6, 0.6, 0.125]) 78.03290939331055
26158282 [8, 0.6, 0.6, 0.125]

In [ ]:
a_example :  (21, [16, 0.2, 0.2, 0.0625]) 0.04893994331359863
b_read_on :  (5822900, [16, 0.2, 0.2, 0.0625]) 34.372737407684326
c_incunabula :  (5648030, [16, 0.2, 0.2, 0.0625]) 122.53631043434143
d_tough_choices :  (4815460, [16, 0.2, 0.2, 0.0625]) 123.71323108673096
e_so_many_books :  (4601808, [16, 0.2, 0.2, 0.0625]) 74.2503387928009
f_libraries_of_the_world :  (5240161, [16, 0.2, 0.2, 0.0625]) 59.03453779220581
26128380 [16, 0.2, 0.2, 0.0625]
e_so_many_books :  (4602102, [16, 0.2, 0.2, 0.125]) 67.93167114257812
d_tough_choices :  (4815525, [16, 0.2, 0.4, 0.125]) 123.5822319984436
e_so_many_books :  (4602185, [16, 0.2, 0.4, 0.125]) 73.00561332702637
f_libraries_of_the_world :  (5252823, [16, 0.2, 0.4, 0.25]) 120.10624980926514
26140670 [16, 0.2, 0.4, 0.25]
d_tough_choices :  (4815590, [16, 0.2, 0.6, 0.0625]) 132.14556336402893
e_so_many_books :  (4602286, [16, 0.2, 0.6, 0.0625]) 97.29728055000305
c_incunabula :  (5648333, [16, 0.4, 0.2, 0.25]) 120.02851986885071
26140890 [16, 0.4, 0.2, 0.25]
c_incunabula :  (5649700, [16, 0.4, 0.4, 0.0625]) 120.24325942993164
f_libraries_of_the_world :  (5258093, [16, 0.4, 0.4, 0.0625]) 120.57875037193298
26148315 [16, 0.4, 0.4, 0.0625]
d_tough_choices :  (4815655, [16, 0.4, 0.4, 0.125]) 122.60754895210266
d_tough_choices :  (4815720, [16, 0.6, 0.2, 0.25]) 127.76291179656982
e_so_many_books :  (4602379, [16, 0.6, 0.2, 0.25]) 82.34583020210266
f_libraries_of_the_world :  (5302911, [16, 0.6, 0.2, 0.25]) 120.28335070610046
26189678 [16, 0.6, 0.2, 0.25]
c_incunabula :  (5651005, [16, 0.6, 0.6, 0.25]) 121.54233479499817